# Counterfactual explanations with one-hot encoded categorical variables

Real world machine learning applications often handle data with categorical variables. Explanation methods which rely on perturbations of the input features need to make sure those perturbations are meaningful and capture the underlying structure of the data. This becomes tricky for categorical features. For instance random perturbations across possible categories or enforcing a ranking between categories based on frequency of occurrence in the training data do not capture this structure. Our method captures the relation between categories of a variable numerically through the context given by the other features in the data and/or the predictions made by the model. First it captures the pairwise distances between categories and then applies multi-dimensional scaling. More details about the method can be found in the [documentation](https://docs.seldon.io/projects/alibi/en/stable/methods/CFProto.html). The example notebook illustrates this approach on the *adult* dataset, which contains a mixture of categorical and numerical features used to predict whether a person's income is above or below $50k.

<div class="alert alert-info">
Note
    
To enable support for CounterfactualProto, you may need to run
    
```bash
pip install alibi[tensorflow]
```

</div>

In [1]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from time import time
from alibi.datasets import fetch_adult
from alibi.explainers import CounterfactualProto
from alibi.utils import ohe_to_ord, ord_to_ohe

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False

TF version:  2.12.1
Eager execution enabled:  False


## Load adult dataset

The `fetch_adult` function returns a `Bunch` object containing the features, the targets, the feature names and a mapping of the categories in each categorical variable.

In [2]:
adult = fetch_adult()
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map_tmp = adult.category_map
target_names = adult.target_names

Define shuffled training and test set:

In [3]:
def set_seed(s=0):
    np.random.seed(s)
    tf.random.set_seed(s)

In [4]:
set_seed()
data_perm = np.random.permutation(np.c_[data, target])
X = data_perm[:,:-1]
y = data_perm[:,-1]

In [5]:
idx = 30000
y_train, y_test = y[:idx], y[idx+1:]

Reorganize data so categorical features come first:

In [6]:
X = np.c_[X[:, 1:8], X[:, 11], X[:, 0], X[:, 8:11]]

Adjust `feature_names` and `category_map` as well:

In [7]:
feature_names = feature_names[1:8] + feature_names[11:12] + feature_names[0:1] + feature_names[8:11]
print(feature_names)

['Workclass', 'Education', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country', 'Age', 'Capital Gain', 'Capital Loss', 'Hours per week']


In [8]:
category_map = {}
for i, (_, v) in enumerate(category_map_tmp.items()):
    category_map[i] = v

Create a dictionary with as keys the categorical columns and values the number of categories for each variable in the dataset:

In [9]:
cat_vars_ord = {}
n_categories = len(list(category_map.keys()))
for i in range(n_categories):
    cat_vars_ord[i] = len(np.unique(X[:, i]))
print(cat_vars_ord)

{0: 9, 1: 7, 2: 4, 3: 9, 4: 6, 5: 5, 6: 2, 7: 11}


Since we will apply one-hot encoding (OHE) on the categorical variables, we convert `cat_vars_ord` from the ordinal to OHE format. `alibi.utils.mapping` contains utility functions to do this. The keys in `cat_vars_ohe` now represent the first column index for each one-hot encoded categorical variable. This dictionary will later be used in the counterfactual explanation.

In [10]:
cat_vars_ohe = ord_to_ohe(X, cat_vars_ord)[1]
print(cat_vars_ohe)

{0: 9, 9: 7, 16: 4, 20: 9, 29: 6, 35: 5, 40: 2, 42: 11}


## Preprocess data

Scale numerical features between -1 and 1:

In [11]:
X_num = X[:, -4:].astype(np.float32, copy=False)
xmin, xmax = X_num.min(axis=0), X_num.max(axis=0)
rng = (-1., 1.)
X_num_scaled = (X_num - xmin) / (xmax - xmin) * (rng[1] - rng[0]) + rng[0]

Apply OHE to categorical variables:

In [12]:
X_cat = X[:, :-4].copy()
ohe = OneHotEncoder(categories='auto', sparse=False).fit(X_cat)
X_cat_ohe = ohe.transform(X_cat)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


Combine numerical and categorical data:

In [13]:
X = np.c_[X_cat_ohe, X_num_scaled].astype(np.float32, copy=False)
X_train, X_test = X[:idx, :], X[idx+1:, :]
print(X_train.shape, X_test.shape)

(30000, 57) (2560, 57)


## Train classifier

In [36]:
# Select one of the below classifiers.
from xgboost import XGBClassifier
clf = XGBClassifier(min_child_weight=0.5, max_depth=3, gamma=0.2)
# clf = LogisticRegression(C=10)
# clf = DecisionTreeClassifier(max_depth=10, min_samples_split=5)
#clf = RandomForestClassifier(max_depth=15, min_samples_split=10, n_estimators=50)

# Fit the classifier.
clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=0.5, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [53]:
def predict_fn(X):
    # The predict_proba method of the pipeline returns an array of shape (n_samples, 2)
    # Return both columns as the CounterfactualProto explainer expects a probability for each class
    pred_proba = clf.predict_proba(X)
    return np.hstack([1 - pred_proba[:, 1].reshape(-1, 1), pred_proba[:, 1].reshape(-1, 1)])

from sklearn.metrics import f1_score
acc = f1_score(y_true=y_test, y_pred=predict_fn(X_test).argmax(axis=1))
print("Accuracy: %.3f" % acc)

Accuracy: 0.718


## Generate counterfactual

Original instance:

In [16]:
X = X_test[0].reshape((1,) + X_test[0].shape)

Initialize counterfactual parameters. The feature perturbations are applied in the numerical feature space, after transforming the categorical variables to numerical features. As a result, the dimensionality and values of `feature_range` are defined in the numerical space.

In [18]:
shape = X.shape
beta = .01
c_init = 1.
c_steps = 5
max_iterations = 500
rng = (-1., 1.)  # scale features between -1 and 1
rng_shape = (1,) + data.shape[1:]
feature_range = ((np.ones(rng_shape) * rng[0]).astype(np.float32), 
                 (np.ones(rng_shape) * rng[1]).astype(np.float32))

Initialize explainer:

In [19]:
def set_seed(s=0):
    np.random.seed(s)
    tf.random.set_seed(s)

Fit explainer. `d_type` refers to the distance metric used to convert the categorical to numerical values. Valid options are `abdm`, `mvdm` and `abdm-mvdm`. `abdm` infers the distance between categories of the same variable from the context provided by the other variables. This requires binning of the numerical features as well. `mvdm` computes the distance using the model predictions, and `abdm-mvdm` combines both methods. More info on both distance measures can be found in the [documentation](https://docs.seldon.io/projects/alibi/en/stable/methods/CFProto.html).

Helper function to more clearly describe explanations:

In [30]:
def describe_instance(X, explanation, eps=1e-2):
    print('Original instance: {}  -- proba: {}'.format(target_names[explanation.orig_class],
                                                       explanation.orig_proba[0]))
    print('Counterfactual instance: {}  -- proba: {}'.format(target_names[explanation.cf['class']],
                                                             explanation.cf['proba'][0]))
    print('\nCounterfactual perturbations...')
    print('\nCategorical:')
    X_orig_ord = ohe_to_ord(X, cat_vars_ohe)[0]
    X_cf_ord = ohe_to_ord(explanation.cf['X'], cat_vars_ohe)[0]
    delta_cat = {}
    for i, (_, v) in enumerate(category_map.items()):
        cat_orig = v[int(X_orig_ord[0, i])]
        cat_cf = v[int(X_cf_ord[0, i])]
        if cat_orig != cat_cf:
            delta_cat[feature_names[i]] = [cat_orig, cat_cf]
    if delta_cat:
        for k, v in delta_cat.items():
            print('{}: {}  -->   {}'.format(k, v[0], v[1]))
    print('\nNumerical:')
    delta_num = X_cf_ord[0, -4:] - X_orig_ord[0, -4:]
    n_keys = len(list(cat_vars_ord.keys()))
    for i in range(delta_num.shape[0]):
        if np.abs(delta_num[i]) > eps:
            print('{}: {:.2f}  -->   {:.2f}'.format(feature_names[i+n_keys],
                                            X_orig_ord[0,i+n_keys],
                                            X_cf_ord[0,i+n_keys]))

In [31]:
describe_instance(X, explanation)

Original instance: <=50K  -- proba: [0.779083   0.22091702]
Counterfactual instance: >50K  -- proba: [0.3595787 0.6404213]

Counterfactual perturbations...

Categorical:
Education: Associates  -->   Bachelors

Numerical:


By obtaining a higher level of education the income is predicted to be above $50k.

In [41]:
set_seed()
cf = CounterfactualProto(predict_fn,
                         shape,
                         beta=beta,
                         cat_vars=cat_vars_ohe,
                         ohe=True,  # OHE flag
                         max_iterations=max_iterations,
                         feature_range=feature_range,
                         c_init=c_init,
                         c_steps=c_steps
                        )

In [42]:
cf.fit(X_train, d_type='abdm', disc_perc=[25, 50, 75]);

The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 

In [56]:
y_pred=predict_fn(X_test).argmax(axis=1)
instances = X_test[y_pred == 1]

import time
# List to store counterfactuals
counterfactuals = []

start_time = time.time()
# Loop through each instance and generate counterfactual


for instance in instances:
    explanation = cf.explain(instance.reshape(1, -1))
    # Check if a counterfactual was found
    if explanation.cf is not None:
        counterfactuals.append(explanation.cf['X'])
        describe_instance(X, explanation)
    else:
        # You can append a placeholder or simply skip
        # Here, I'm appending None to indicate no counterfactual was found for this instance
        counterfactuals.append(None)
        
# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

No counterfactual found!


Original instance: >50K  -- proba: [0.4741686 0.5258314]
Counterfactual instance: <=50K  -- proba: [0.6037574  0.39624265]

Counterfactual perturbations...

Categorical:
Marital Status: Never-Married  -->   Married
Relationship: Own-child  -->   Husband

Numerical:
Age: -0.89  -->   -0.10
Hours per week: -0.61  -->   -0.10


No counterfactual found!
No counterfactual found!
No counterfactual found!


Original instance: >50K  -- proba: [0.41468304 0.58531696]
Counterfactual instance: <=50K  -- proba: [0.55429494 0.44570506]

Counterfactual perturbations...

Categorical:
Marital Status: Never-Married  -->   Married
Occupation: Blue-Collar  -->   Sales
Relationship: Own-child  -->   Husband

Numerical:
Age: -0.89  -->   -0.45
Hours per week: -0.61  -->   -0.16
Original instance: >50K  -- proba: [0.23521066 0.76478934]
Counterfactual instance: <=50K  -- proba: [0.6633158 0.3366842]

Counterfactual perturbations...

Categorical:
Education: High School grad  -->   Bachelors
Marital Status: Never-Married  -->   Married
Occupation: Blue-Collar  -->   Service
Relationship: Own-child  -->   Husband

Numerical:
Age: -0.89  -->   0.21
Hours per week: -0.61  -->   -0.20


KeyboardInterrupt: 

## Change the categorical distance metric

Instead of `abdm`, we now use `mvdm` as our distance metric.

In [26]:
set_seed()
cf.fit(X_train, d_type='mvdm')
explanation = cf.explain(X)
describe_instance(X, explanation)

Original instance: <=50K  -- proba: [0.70744723 0.29255277]
Counterfactual instance: >50K  -- proba: [0.38161737 0.61838263]

Counterfactual perturbations...

Categorical:
Education: Associates  -->   Bachelors

Numerical:


The same conclusion hold using a different distance metric.

## Use k-d trees to build prototypes

We can also use *k-d trees* to build class prototypes to guide the counterfactual to nearby instances in the counterfactual class as described in [Interpretable Counterfactual Explanations Guided by Prototypes](https://arxiv.org/abs/1907.02584). 

In [27]:
use_kdtree = True
theta = 10.  # weight of prototype loss term

Initialize, fit and explain instance:

In [28]:
set_seed()
X = X_test[7].reshape((1,) + X_test[0].shape)
cf = CounterfactualProto(nn,
                         shape,
                         beta=beta,
                         theta=theta,
                         cat_vars=cat_vars_ohe,
                         ohe=True,
                         use_kdtree=use_kdtree,
                         max_iterations=max_iterations,
                         feature_range=feature_range,
                         c_init=c_init,
                         c_steps=c_steps
                        )
cf.fit(X_train, d_type='abdm')
explanation = cf.explain(X)
describe_instance(X, explanation)

Original instance: <=50K  -- proba: [0.5211548  0.47884512]
Counterfactual instance: >50K  -- proba: [0.49958408 0.500416  ]

Counterfactual perturbations...

Categorical:

Numerical:
Age: -0.53  -->   -0.51


By slightly increasing the age of the person the income would be predicted to be above $50k.

## Use an autoencoder to build prototypes

Another option is to use an autoencoder to guide the perturbed instance to the counterfactual class. We define and train the autoencoder:

In [57]:
def ae_model():
    # encoder
    x_in = Input(shape=(57,))
    x = Dense(60, activation='relu')(x_in)
    x = Dense(30, activation='relu')(x)
    x = Dense(15, activation='relu')(x)
    encoded = Dense(10, activation=None)(x)
    encoder = Model(x_in, encoded)
    
    # decoder
    dec_in = Input(shape=(10,))
    x = Dense(15, activation='relu')(dec_in)
    x = Dense(30, activation='relu')(x)
    x = Dense(60, activation='relu')(x)
    decoded = Dense(57, activation=None)(x)
    decoder = Model(dec_in, decoded)
    
    # autoencoder = encoder + decoder
    x_out = decoder(encoder(x_in))
    autoencoder = Model(x_in, x_out)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder, encoder, decoder

In [59]:
set_seed()
ae, enc, dec = ae_model()
ae.summary()
ae.fit(X_train, X_train, batch_size=128, epochs=100, validation_data=(X_test, X_test), verbose=1)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 57)]              0         
                                                                 
 model_4 (Functional)        (None, 10)                5935      
                                                                 
 model_5 (Functional)        (None, 57)                5982      
                                                                 
Total params: 11,917
Trainable params: 11,917
Non-trainable params: 0
_________________________________________________________________
Train on 30000 samples, validate on 2560 samples
Epoch 1/100
30000/30000 [==============================] - 2s 51us/sample - loss: 0.0684 - val_loss: 0.0434
Epoch 2/100
30000/30000 [==============================] - 1s 22us/sample - loss: 0.0369 - val_loss: 0.0333
Epoch 3/100
30000/30000 [=============================

30000/30000 [==============================] - 1s 21us/sample - loss: 0.0109 - val_loss: 0.0111
Epoch 69/100
30000/30000 [==============================] - 1s 23us/sample - loss: 0.0108 - val_loss: 0.0110
Epoch 70/100
30000/30000 [==============================] - 1s 21us/sample - loss: 0.0108 - val_loss: 0.0110
Epoch 71/100
30000/30000 [==============================] - 1s 22us/sample - loss: 0.0108 - val_loss: 0.0111
Epoch 72/100
30000/30000 [==============================] - 1s 21us/sample - loss: 0.0108 - val_loss: 0.0111
Epoch 73/100
30000/30000 [==============================] - 1s 21us/sample - loss: 0.0108 - val_loss: 0.0111
Epoch 74/100
30000/30000 [==============================] - 1s 21us/sample - loss: 0.0108 - val_loss: 0.0110
Epoch 75/100
30000/30000 [==============================] - 1s 19us/sample - loss: 0.0107 - val_loss: 0.0110
Epoch 76/100
30000/30000 [==============================] - 1s 20us/sample - loss: 0.0107 - val_loss: 0.0110
Epoch 77/100
30000/30000 [======

Weights for the autoencoder and prototype loss terms:

In [61]:
beta = .1  # L1
gamma = 10.  # autoencoder
theta = .1  # prototype

Initialize, fit and explain instance:

In [66]:
set_seed()
X = X_test[19].reshape((1,) + X_test[0].shape)
cf = CounterfactualProto(predict_fn,
                         shape,
                         beta=beta,
                         enc_model=enc,
                         ae_model=ae,
                         gamma=gamma,
                         theta=theta,
                         cat_vars=cat_vars_ohe,
                         ohe=True,
                         max_iterations=max_iterations,
                         feature_range=feature_range,
                         c_init=c_init,
                         c_steps=c_steps,
                         use_kdtree = True
                        )
cf.fit(X_train, d_type='abdm')


Both an encoder and k-d trees enabled. Using the encoder for the prototype loss term.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized

CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 10.0,
              'theta': 0.1,
              'cat_vars': {
                            0: 9,
                            9: 7,
                            16: 4,
                            20: 9,
                            29: 6,
                            35: 5,
                            40: 2,
                            42: 11}
                          ,
              'ohe': True,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 500,
              'c_init': 1.0,
              'c_steps': 5,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (array([[-1., -1., -1., -1.,

In [67]:
y_pred=predict_fn(X_test).argmax(axis=1)
instances = X_test[y_pred == 1]

import time
# List to store counterfactuals
counterfactuals = []


# Loop through each instance and generate counterfactual


for instance in instances:
    start_time = time.time()
    explanation = cf.explain(instance.reshape(1, -1))
    # Check if a counterfactual was found
    if explanation.cf is not None:
        counterfactuals.append(explanation.cf['X'])
        describe_instance(instance.reshape(1, -1), explanation)
    else:
        # You can append a placeholder or simply skip
        # Here, I'm appending None to indicate no counterfactual was found for this instance
        counterfactuals.append(None)
    end_time = time.time()
# Calculate the elapsed time
    elapsed_time = end_time - start_time
    print(elapsed_time)


Original instance: >50K  -- proba: [0.09381664 0.90618336]
Counterfactual instance: <=50K  -- proba: [0.6888168 0.3111832]

Counterfactual perturbations...

Categorical:

Numerical:
Capital Loss: -0.13  -->   -0.18
83.16250443458557


KeyboardInterrupt: 

# with rl

In [68]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy
from typing import List, Tuple, Dict, Callable

import tensorflow as tf
import tensorflow.keras as keras

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from alibi.explainers import CounterfactualRLTabular, CounterfactualRL
from alibi.datasets import fetch_adult
from alibi.models.tensorflow import HeAE
from alibi.models.tensorflow import Actor, Critic
from alibi.models.tensorflow import ADULTEncoder, ADULTDecoder
from alibi.explainers.cfrl_base import Callback
from alibi.explainers.backends.cfrl_tabular import get_he_preprocessor, get_statistics, \
    get_conditional_vector, apply_category_mapping

In [ ]:
explainer = CounterfactualRLTabular(predictor=predictor,
                                    encoder=heae.encoder,
                                    decoder=heae.decoder,
                                    latent_dim=LATENT_DIM,
                                    encoder_preprocessor=heae_preprocessor,
                                    decoder_inv_preprocessor=heae_inv_preprocessor,
                                    coeff_sparsity=COEFF_SPARSITY,
                                    coeff_consistency=COEFF_CONSISTENCY,
                                    category_map=adult.category_map,
                                    feature_names=adult.feature_names,
                                    ranges=ranges,
                                    immutable_features=immutable_features,
                                    train_steps=TRAIN_STEPS,
                                    batch_size=BATCH_SIZE,
                                    backend="tensorflow")